# AWS Bedrock LLMManager Example Notebook

This notebook demonstrates how to use the LLMManager class to interact with AWS Bedrock's Converse API with built-in resilience and failover capabilities. The LLMManager provides:

1. **Multi-region support** - Try models across different AWS regions
2. **Model fallback** - Automatically switch to alternative models if the primary choice fails
3. **Cross-Region Inference (CRIS) optimization** - Automatically use CRIS when available
4. **Error handling** - Gracefully handle API errors with appropriate fallbacks
5. **Flexible authentication** - Support for AWS CLI profiles, access keys, or IAM roles

## Import the LLMManager class and other required modules

In [ ]:
import time
import logging
import json

# Import from our package
from src import LLMManager, Fields, Roles, PerformanceConfig

## Configure AWS Profile

Set your AWS CLI profile name below. This profile should have permissions to access AWS Bedrock.

In [ ]:
# Replace with your AWS CLI profile name
aws_profile = "default"

## Basic Initialization of LLMManager

Create an instance of the LLMManager with the AWS profile and default settings.

In [ ]:
# Initialize LLMManager with AWS profile
llm_manager = LLMManager(
    profile_name=aws_profile,
    regions=["us-east-1", "us-west-2", "eu-west-1"],  # Try multiple regions in order of preference
    model_ids=["anthropic.claude-3-sonnet-20240229-v1:0", "anthropic.claude-3-haiku-20240307-v1:0"],  # List models in order of preference
    log_level=logging.INFO
)

## Send a Simple Text Prompt

Let's send a simple prompt to the LLM and see the response.

In [ ]:
# Define the prompt
prompt = "Write a paragraph on the planets of the solar system."

# Create a text message using the helper method
messages = [llm_manager.create_text_message(prompt)]

# Send the prompt to the model
try:
    response = llm_manager.converse(messages=messages)
    
    # Print the response text
    print("Model response text:")
    print(response.get_content_text())
    
    print("\nMetadata:")
    print(f"Model used: {response.model_id}")
    print(f"Region used: {response.region}")
    print(f"Used CRIS: {response.is_cris}")
    print(f"Execution time: {response.execution_time:.3f} seconds")
    print(f"Input tokens: {response.get_input_tokens()}")
    print(f"Output tokens: {response.get_output_tokens()}")
    print(f"Total tokens: {response.get_total_tokens()}")
    
except Exception as e:
    print(f"Error: {str(e)}")

## Using System Prompts

System prompts can be used to provide context or instructions to the model.

In [ ]:
# Create a system message using the helper method
system = [llm_manager.create_system_message(
    "You are a helpful AI assistant specializing in astronomy. "
    "Provide concise and accurate information."
)]

# Define the prompt
prompt = "What are the most interesting facts about Jupiter?"
messages = [llm_manager.create_text_message(prompt)]

# Send the prompt with the system message
try:
    response = llm_manager.converse(
        messages=messages,
        system=system
    )
    
    print("Response with system prompt:")
    print(response.get_content_text())
    
except Exception as e:
    print(f"Error: {str(e)}")

## Working with Different Inference Parameters

The LLMManager allows you to customize inference parameters such as temperature, max tokens, top_p, and stop sequences.

In [ ]:
# Define the prompt
prompt = "Write a creative short story about space exploration."
messages = [llm_manager.create_text_message(prompt)]

# Define inference config with different parameters
inference_config = {
    Fields.TEMPERATURE: 0.9,  # Higher temperature for more creative output
    Fields.MAX_TOKENS: 300,   # Limit the response length
    Fields.TOP_P: 0.95,       # Sample from a wider range of tokens
    Fields.STOP_SEQUENCES: ["The End"]  # Stop generation at "The End"
}

# Send the prompt with custom inference parameters
try:
    response = llm_manager.converse(
        messages=messages,
        inference_config=inference_config
    )
    
    print("Response with custom inference parameters:")
    print(response.get_content_text())
    print(f"\nStop reason: {response.get_stop_reason()}")
    print(f"Output tokens: {response.get_output_tokens()}")
    
except Exception as e:
    print(f"Error: {str(e)}")

## Using Different Performance Settings

You can optimize for latency using the performance_config parameter.

In [ ]:
# Define the prompt
prompt = "What is the capital of France?"
messages = [llm_manager.create_text_message(prompt)]

# Define performance config for latency optimization
performance_config = {
    Fields.LATENCY: PerformanceConfig.OPTIMIZED
}

# Send the prompt with optimized latency
try:
    response = llm_manager.converse(
        messages=messages,
        performance_config=performance_config
    )
    
    print("Response with optimized latency:")
    print(response.get_content_text())
    print(f"\nLatency: {response.get_latency_ms()} ms")
    
except Exception as e:
    print(f"Error: {str(e)}")

## Specifying Different Models and Regions

You can override the default models and regions when calling the converse method.

In [ ]:
# Define the prompt
prompt = "What are three benefits of quantum computing?"
messages = [llm_manager.create_text_message(prompt)]

# Specify different models and regions
alternative_models = ["amazon.nova-pro-v1:0", "anthropic.claude-3-haiku-20240307-v1:0"]
alternative_regions = ["us-west-2", "us-east-1"]

try:
    response = llm_manager.converse(
        messages=messages,
        model_ids=alternative_models,
        regions=alternative_regions
    )
    
    print("Response with alternative models and regions:")
    print(response.get_content_text())
    print(f"\nModel used: {response.model_id}")
    print(f"Region used: {response.region}")
    print(f"Used CRIS: {response.is_cris}")
    
except Exception as e:
    print(f"Error: {str(e)}")

## Using Cross-Region Inference Profiles

The LLMManager automatically uses Cross-Region Inference Service (CRIS) profiles when available. Let's explicitly create an instance that focuses on CRIS-enabled regions.

In [ ]:
# Create a LLMManager instance focused on regions that support CRIS
cris_manager = LLMManager(
    profile_name=aws_profile,
    # These regions are common source regions for CRIS profiles
    regions=["us-east-1", "us-east-2", "us-west-2", "eu-central-1", "eu-west-1", "ap-northeast-1"],
    # Models that are widely available through CRIS
    model_ids=["anthropic.claude-3-sonnet-20240229-v1:0", "anthropic.claude-3-haiku-20240307-v1:0"]
)

# Define the prompt
prompt = "Summarize the key benefits of AWS Cross-Region Inference Service."
messages = [cris_manager.create_text_message(prompt)]

try:
    response = cris_manager.converse(messages=messages)
    
    print("Response from CRIS-focused manager:")
    print(response.get_content_text())
    print(f"\nModel used: {response.model_id}")
    print(f"Region used: {response.region}")
    print(f"Used CRIS: {response.is_cris}")
    if response.is_cris:
        print(f"The model is being accessed through CRIS!")
    
except Exception as e:
    print(f"Error: {str(e)}")

## Error Handling and Fallback Mechanisms

The LLMManager automatically handles errors and falls back to alternative models and regions. Let's see how it behaves when we provide an invalid model ID.

In [ ]:
# Define the prompt
prompt = "What is AWS Bedrock?"
messages = [llm_manager.create_text_message(prompt)]

# Specify a non-existent model ID first, followed by a valid one
test_models = ["invalid.model-v1:0", "anthropic.claude-3-haiku-20240307-v1:0"]

try:
    response = llm_manager.converse(
        messages=messages,
        model_ids=test_models
    )
    
    print("Response after fallback:")
    print(response.get_content_text())
    print(f"\nModel used: {response.model_id}")
    print(f"Region used: {response.region}")
    
except Exception as e:
    print(f"Error: {str(e)}")

## Working with Additional Features (Tool Config, Guardrails, etc.)

The LLMManager supports additional features such as tool configuration, guardrails, and request metadata.

In [ ]:
# Define a request with metadata
prompt = "What's the capital of Germany?"
messages = [llm_manager.create_text_message(prompt)]

# Define request metadata
request_metadata = {
    "requestId": f"test-{int(time.time())}",
    "context": "geography-query"
}

try:
    response = llm_manager.converse(
        messages=messages,
        request_metadata=request_metadata
    )
    
    print("Response with request metadata:")
    print(response.get_content_text())
    
except Exception as e:
    print(f"Error: {str(e)}")

## Working with Images (Multimodal Models)

If you have image bytes, you can use the create_image_message helper to create messages with images for multimodal models.

In [ ]:
# This is a placeholder to demonstrate the image message creation
# In a real implementation, you would load an actual image file
'''
# Example: Load an image file
with open("path/to/image.jpg", "rb") as f:
    image_bytes = f.read()

# Create an image message
image_message = llm_manager.create_image_message(
    image_bytes=image_bytes,
    text="What's in this image?",
    image_format="jpeg"
)

# Send the message to a multimodal model
multimodal_models = ["anthropic.claude-3-sonnet-20240229-v1:0"]
response = llm_manager.converse(
    messages=[image_message],
    model_ids=multimodal_models
)

print(response.get_content_text())
'''

## Conclusion

This notebook demonstrated the capabilities of the LLMManager class for interacting with AWS Bedrock models. The key features include:

1. Automatic model and region fallbacks for improved reliability
2. Cross-Region Inference Service (CRIS) optimization
3. Support for various inference parameters and configurations
4. Helper methods for creating messages with different content types
5. Comprehensive error handling and response metadata

Use the LLMManager to build robust applications that can leverage AWS Bedrock's powerful LLM capabilities with built-in resilience and optimizations.